In [ ]:
import $ivy.`io.github.quafadas::scautable:0.0.33`
import $ivy.`org.apache.commons:commons-math3:3.6.1`
import $ivy.`io.github.quafadas::dedav4s:0.10.0-RC2`
import $exec.`utils2`
// heatmap plotting
import $exec.`heatmap2`
import io.github.quafadas.table.*
import viz.PlotTargets.almond
import org.apache.commons.math3.distribution.EnumeratedIntegerDistribution

Compiling /Users/simon/Code/makemore/utils2.sc


This notebook adopts a "bayesian" or "classical" approach, following the exact intuition you would follow, if you were attempting to solve this problem yourself. 

1. collect the bigrams. 
2. group them by first character
3. for each group, create a probability distribution of the second character

Then continute with generation as before.

In [ ]:
private val chars = '.' +: ('a' to 'z').toVector
private val charsIndex = (0 to 26).toVector
val charsMap = chars.zip(charsIndex).toMap
val i2c = charsIndex.zip(chars).toMap

val smooth = true
private val combinations = for {
  a <- chars
  b <- chars
} yield s"$a$b"

private val data = CSV.pwd("names.txt")

private val bookended = LazyList.from(
  data
    .addColumn["Bookend", String](s => s".${s.name}.")
    .addColumn["Pairs", Seq[String]](s => s.Bookend.sliding(2).toSeq)
)
bookended.take(5).ptbln

private val pairSet: Map[String, Int] = bookended
  .flatMap(_.Pairs)
  .groupMapReduce(identity)(_ => 1)(_ + _)

val missing: Set[String] = combinations.toSet.diff(pairSet.keys.toSet)

println("missing combinations")
println(missing)

private val completePairs: Seq[(String, Int)] = pairSet.toSeq ++ missing.map(char => (char, if(smooth) 1 else 0))

private val grouped: Map[Char, Seq[(String, Int)]] = completePairs.groupBy(d => d._1.head)

val normalised: Map[Char, EnumeratedIntegerDistribution] = grouped.view.mapValues { v =>
  val sortedByChar = v.sortBy(_._1.last)
  val col = sortedByChar.map(_._2.toDouble)
  new EnumeratedIntegerDistribution(
    charsIndex.toArray,
    col.toArray
  )
}.toMap
heatmap(completePairs.seq, "Character bigram counts (not probability weighted)")
heatmap(normalised, charsMap, i2c, "Character bigram (probability weighted to a probability dist for the first Character)")

-- Warning: cmd13.sc:42:37 -----------------------------------------------------
42 |val res13_16 = heatmap(completePairs.seq, "Character bigram counts (not probability weighted)")
   |                       ^^^^^^^^^^^^^^^^^
   |method seq in trait Iterable is deprecated since 2.13.0: Iterable.seq always returns the iterable itself


| |    name|   Bookend|                                   Pairs|
+-+--------+----------+----------------------------------------+
|0|    emma|    .emma.|                List(.e, em, mm, ma, a.)|
|1|  olivia|  .olivia.|        List(.o, ol, li, iv, vi, ia, a.)|
|2|     ava|     .ava.|                    List(.a, av, va, a.)|
|3|isabella|.isabella.|List(.i, is, sa, ab, be, el, ll, la, a.)|
|4|  sophia|  .sophia.|        List(.s, so, op, ph, hi, ia, a.)|
+-+--------+----------+----------------------------------------+
missing combinations
HashSet(qb, bk, qf, jq, cb, kp, qq, xj, gx, zq, vw, bf, wx, fq, fv, tp, qc, fj, wc, qv, gk, vq, jg, sx, bw, qg, mq, jx, bq, pw, vm, wj, fc, xk, qd, qk, zf, vx, fx, fm, mx, jz, qp, mg, bg, cn, tk, vp, vf, bm, xr, wq, qy, wv, gq, px, qh, fd, fp, td, bx, cv, qj, pv, vs, pz, xp, qt, vj, lx, bz, pd, bp, xv, cm, .., gc, fb, xq, gp, hx, xg, wp, vt, qx, jf, bv, vc, cf, cw, qz, qn, tq, dx, vz, vg, kg, pq, kq, kx, dp, pg)


charsMap: Map[Char, Int] = HashMap(
  'e' -> 5,
  'n' -> 14,
  '.' -> 0,
  't' -> 20,
  'a' -> 1,
  'm' -> 13,
  'i' -> 9,
  'v' -> 22,
  'p' -> 16,
  'r' -> 18,
  'w' -> 23,
  'k' -> 11,
  's' -> 19,
  'x' -> 24,
  'j' -> 10,
  'y' -> 25,
  'u' -> 21,
  'f' -> 6,
  'q' -> 17,
  'b' -> 2,
  'g' -> 7,
  'l' -> 12,
  'c' -> 3,
  'h' -> 8,
  'o' -> 15,
  'z' -> 26,
  'd' -> 4
)
i2c: Map[Int, Char] = HashMap(
  0 -> '.',
  5 -> 'e',
  10 -> 'j',
  14 -> 'n',
  20 -> 't',
  1 -> 'a',
  6 -> 'f',
  21 -> 'u',
  9 -> 'i',
  13 -> 'm',
  2 -> 'b',
  17 -> 'q',
  22 -> 'v',
  12 -> 'l',
  7 -> 'g',
  3 -> 'c',
  18 -> 'r',
  16 -> 'p',
  11 -> 'k',
  8 -> 'h',
  19 -> 's',
  4 -> 'd',
  15 -> 'o',
  24 -> 'x',
  25 -> 'y',
  26 -> 'z',
  23 -> 'w'
)
smooth: Boolean = true
missing: Set[String] = HashSet(
  "qb",
  "bk",
  "qf",
  "jq",
  "cb",
  "kp",
  "qq",
  "xj",
  "gx",
  "zq",
  "vw",
  "bf",
  "wx",
  "fq",
  "fv",
  "tp",
  "qc",
  "fj",
  "wc",
  "qv",
  "gk",
  "vq",
  "jg",
  "sx",
  

In [ ]:
println("Moderately less rubbish name generator")
for (i <- 0 to 5) {
  println(generator(normalised, i2c).mkString(""))
}

val someNames = Seq("simon", "isolde", "arlo", "axel", "zzz", "zzzzzzzzz", "christoph")

checkWords(someNames, normalised, charsMap).ptbln

Moderately less rubbish name generator
n
alelon
itariyorona
amitrelin
begin
kyi
| |     word|    totalLikelihood|                  prob|      avg_prbability|
+-+---------+-------------------+----------------------+--------------------+
|0|    simon|-13.819601957026752|  9.959169593076386E-7| 0.09993183325971156|
|1|   isolde|-19.574725646714104|  3.153585912319969E-9| 0.06103002162438039|
|2|     arlo| -12.98095674729182|  2.303785894249425E-6| 0.07455699970232603|
|3|     axel|-14.384574433386492|  5.660550888326395E-7| 0.05630821235696359|
|4|      zzz|-14.201617633148048|  6.796977430230372E-7| 0.02871302552050607|
|5|zzzzzzzzz| -38.06098679254235|2.9534080902564186E-17|0.022234754877491308|
|6|christoph| -24.03983877818535| 3.627694217823434E-11| 0.09035726299763763|
+-+---------+-------------------+----------------------+--------------------+


someNames: Seq[String] = List(
  "simon",
  "isolde",
  "arlo",
  "axel",
  "zzz",
  "zzzzzzzzz",
  "christoph"
)